<a href="https://www.kaggle.com/code/yasirmehmood2/flood-prediction-model?scriptVersionId=178542942" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
import pandas as pd

from sklearn.tree import DecisionTreeClassifier
from sklearn.multioutput import MultiOutputRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier

import numpy as np

from sklearn.metrics import accuracy_score

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
# importing CSV with training data
file_path = '/kaggle/input/playground-series-s4e5/train.csv'
    # Read the CSV file into a DataFrame
df = pd.read_csv(file_path)

X=df.copy()

X.drop(columns=['FloodProbability'],inplace=True)

y=df[['FloodProbability']]

In [ ]:
#splitting the data into training data and test data ( aorund 20% of data will be used for testing the results whereas 80% for training the model)

#X_train contains training data
# y_train cotains result set for training data  ( both X_train and y_train will train the model)
# X_test contains the test data
# y_test contains test expected Results
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

X_train_original=X_train
X_test_original=X_test
y_train_original=y_train
y_test_original=y_test

In [ ]:
#---------------------------------MultiOutputRegressorModel----------------------------------------------
#model = MultiOutputRegressor(DecisionTreeRegressor())   # Not recommended as it requires 2 d array for Output variable ( Y)
#.......................................................................................................
#----------------------------------Liner Regression -----------------------------------------------

# Regression model is used when result is a number like 5,6,9,500 etc.
#model = LinearRegression()   #its results have less accuracy as compared to Plynomial regression model

#----------------------------------Polynomal Regression--------------------------
degree = 2  # Degree of the polynomial
X_test_columnlist=X_train.columns

poly_features = PolynomialFeatures(degree=degree)
X_train = poly_features.fit_transform(X_train)

X_test = poly_features.transform(X_test)

model = LinearRegression()   #R-squared (R²) Score: 0.8461306949654057

#polynomial regression, decision trees, random forests, or gradient boosting


#----------------------- Ridge Regression model ------------------------

# Note : Result of Ridge Model and Polynomial Regression model is same , therefore using any one of both is OK


#  Sample Code is given below , whihc can be uncommented if any one wants to use this model


#    from sklearn.linear_model import Ridge

#     model =Ridge(alpha=1.0, fit_intercept=True, copy_X=True, max_iter=None, tol=0.001, solver='auto', random_state=None)


In [ ]:
# Reshape y_train to a 1D array
y_train = np.ravel(y_train)

#training the model
model.fit(X_train,y_train)


In [ ]:
# Predicting the values on the basis of sample
prediction=model.predict(X_test)


mae = mean_absolute_error(y_test, prediction)
mse = mean_squared_error(y_test, prediction)
rmse = mean_squared_error(y_test, prediction, squared=False)  # Compute RMSE from MSE
r2 = r2_score(y_test, prediction)

print("Mean Absolute Error (MAE):", mae)
print("Mean Squared Error (MSE):", mse)
print("Root Mean Squared Error (RMSE):", rmse)
print("R-squared (R²) Score:", r2)


In [ ]:
# Getting Column Names from Original Input dataframe and extractingn the same columns from polynomial array 

# Get feature names for the polynomial features
feature_names = poly_features.get_feature_names_out(X_train_original.columns)

# Create a DataFrame for polynomial features to easily identify the original features
input_df = pd.DataFrame(X_test_original, columns=feature_names)

# Create a DataFrame for output data ( this dataframe will further be concatenated with Prediction so that accurace of the model could be gauged)
output_df = pd.DataFrame(y_test, columns=['FloodProbability'])  # Replace with appropriate column name
output_df.rename(columns={'FloodProbability': 'FloodProbability_test'}, inplace=True)

# Create a DataFrame for prediction results
prediction_df = pd.DataFrame(prediction, columns=['FloodProbability_Predicted'])  # Assuming prediction is a 1D array



#writing output in CSV for further asessment 

input_df.to_csv('/kaggle/working/input_FloodPrediction.csv', index=False)
output_df.to_csv('/kaggle/working/output_FloodPrediction.csv', index=False)
prediction_df.to_csv('/kaggle/working/FloodPrediction.csv', index=False)



df_input_csv= pd.read_csv('/kaggle/working/input_FloodPrediction.csv')
df_output_csv= pd.read_csv('/kaggle/working/output_FloodPrediction.csv')
df_prediction_csv= pd.read_csv('/kaggle/working/FloodPrediction.csv')



# Concatenate input, output, and prediction DataFrames along the columns axis
combined_df = pd.concat([df_input_csv, df_output_csv,df_prediction_csv], axis=1)



combined_df.to_csv('/kaggle/working/FloodPrediction_final.csv', index=False)



In [ ]:
#producing Results for Submission to Kagal on Kagal provided files

# importing CSV with training data
file_path = '/kaggle/input/playground-series-s4e5/test.csv'

df22=pd.read_csv(file_path)

df22.to_csv('/kaggle/working/Data_before_prediction.csv', index=False)

#defining input and output dataframes
Xa=df22.copy()


degree = 2  # Degree of the polynomial

poly_features = PolynomialFeatures(degree=degree)
Xa = poly_features.fit_transform(Xa)

#predicting results
prediction_submission=model.predict(Xa)

# Create a DataFrame for prediction results
prediction_df_final = pd.DataFrame(prediction_submission, columns=['FloodProbability'])  # Assuming prediction is a 1D array

prediction_df_final.to_csv('/kaggle/working/output_df_results.csv', index=False)


# Create a DataFrame for input data

# Get feature names for the polynomial features
feature_names = poly_features.get_feature_names_out(df22.columns)

# Create a DataFrame for polynomial features to easily identify the original features
input_df = pd.DataFrame(Xa, columns=feature_names)


#input_df = pd.DataFrame(Xa, columns=Xa.columns.tolist())  # Replace with appropriate column names

input_df.to_csv('/kaggle/working/input_df_results.csv', index=False)


df_input_csv_fin= pd.read_csv('/kaggle/working/input_df_results.csv')
df_output_csv_fin= pd.read_csv('/kaggle/working/output_df_results.csv')

df_input_csv_fin=df_input_csv_fin[['id']]
# Concatenate input, output, and prediction DataFrames along the columns axis
combined_df_fin = pd.concat([df_input_csv_fin, df_output_csv_fin], axis=1)


combined_df_fin.to_csv('/kaggle/working/combined_df_fin.csv', index=False)
